In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.express as px
import json
import uuid
pd.set_option('display.float_format','{:.2f}'.format)

In [2]:
esci_data_path = "../../../esci-data/shopping_queries_dataset/"

df_examples = pd.read_parquet(esci_data_path + "shopping_queries_dataset_examples.parquet")
df_examples = df_examples[df_examples.product_locale=="us"]

In [3]:
df_examples.head(20)

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train
5,5,revent 80 cfm,0,B076Q7V5WX,us,E,0,1,train
6,6,revent 80 cfm,0,B075ZBF9HG,us,E,0,1,train
7,7,revent 80 cfm,0,B06W2LB17J,us,E,0,1,train
8,8,revent 80 cfm,0,B07JY1PQNT,us,E,0,1,train
9,9,revent 80 cfm,0,B01MZIK0PI,us,E,0,1,train


In [4]:
df_examples.describe(include='object')

,query,product_id,product_locale,esci_label,split
count,1818825,1818825,1818825,1818825,1818825
unique,97345,1215851,1,4,2
top,tv,B01HFFXLNA,us,E,train
freq,198,120,1818825,1247558,1393063


In [5]:
df_examples.query_id.unique().size

97345

## Query set

A query set has the columns:
 * query_set_id
 * query

Potentially other columns:
 * (head/torso/tail)

There is currently no date corresponding to the query set.
And currently the sampling is not done based on frequency

In [6]:
np.random.seed(10)

In [7]:
# Sample query sets

query_sets = [("sampled_queries", 200), ("top_queries", 20)]

res = []

for query_set_id, n_query_set in query_sets:
    # todo: sampling proportional to frequency
    query_set = np.random.choice(df_examples["query"].unique(), n_query_set, replace=False)

    # todo: perhaps consider product_locale? what about small/large version?

    df = pd.DataFrame({"query": query_set})
    df["query_set_id"] = query_set_id
    res.append(df)
df_query_set = pd.concat(res)
df_query_set.head(10)

,query,query_set_id
0,runtz,sampled_queries
1,trooper bandana shoe,sampled_queries
2,tcl a1x phone case straight talk,sampled_queries
3,bose headphones replacement cord,sampled_queries
4,uniball vision elite,sampled_queries
5,definitely not paid enough for this,sampled_queries
6,raid deep reach fogger,sampled_queries
7,usb camera,sampled_queries
8,reusable produce bags,sampled_queries
9,latex dental dam,sampled_queries


In [8]:
df_query_set.to_csv("query_set.csv", index=False)

## Judgments

The judgments dataset has a row per query instance and document and has the following columns:
 * datetime: date of query/document instance
 * query_id: identifier of query instance
 * query: the query
 * document: identifier of a document result
 * judgment: Here we use the proposed ESCI mapping for DCG: `{"E": 0, "S": 1, "C": 2, "I": 3}`

In [9]:
# Select judgments
# Map esci_label to score
# create judgments per day in range
# create noise in score

label_num = {"E": 0, "S": 1, "C": 2, "I": 3}
label_score = [1, 0.1, 0.01, 0]
label_p_noise = 0.1

def label_to_score(label):
    return label_score[label_num[label]]

df_judge = df_examples[df_examples["query"].isin(set(df_query_set["query"].values))].copy()
df_judge["judgment"] = df_judge.esci_label.apply(lambda x: label_to_score(x))
df_judge["document"] = df_judge.product_id
df_judge = df_judge[["query", "document", "judgment"]].reset_index(drop=True)
df_judge.head(20)

,query,document,judgment
0,$30 roblox gift card not digital,B07RX6FBFR,1.00
1,$30 roblox gift card not digital,B09194H44R,0.00
2,$30 roblox gift card not digital,B08R5N6W6B,0.10
3,$30 roblox gift card not digital,B07Y693ND1,0.00
4,$30 roblox gift card not digital,B07RZ75JW3,0.10
5,$30 roblox gift card not digital,B07RZ74VLR,0.10
6,$30 roblox gift card not digital,B07M9XQ9YB,0.00
7,$30 roblox gift card not digital,B078RJ1KZ6,0.00
8,$30 roblox gift card not digital,B01N6RK9UE,0.00
9,$30 roblox gift card not digital,B016Y2BVKA,1.00


In [10]:
df_judge.to_csv("judgments.csv", index=False)

## Fake Search Algorithms

These are not real search algorithms, rather fake algorithms in order to compute metrics.
An algorithm receives a document result set for a query and returns which documents it keeps
and what is the score that is assigned to them.

In [11]:
import hashlib

def digest_str(s):
    m = hashlib.sha256()
    m.update(str.encode(s))
    return float(m.digest()[0]) / 256

class DropAndTopSearch:
    """
    Randomly drops documents from judgments and returns scores for kept documents.
    Emulates selecting some documents presents in judgments and showing them in top positions.
    """
    def __init__(self, name, p_keep, noise):
        self.name = name
        self.p_keep = p_keep
        self.noise_magnitude = noise
    def run_search(self, df_judgment, date):
        """
        inputs
          df_judgment: query results dataframe for a single query,
            it contains the columns `query`, `document`.
            
        returns additional columns
          keep: whether the document is returned as part of the search
          score: what is the score by which the document is ranked
        """
        seed = df_judgment["query"] + df_judgment.document + self.name
        num = seed.apply(digest_str)
        noise_seed = self.name + date
        noise = (digest_str(noise_seed) - 0.5) * 2.0 * self.noise_magnitude
        df_judgment = df_judgment.copy()
        df_judgment["keep"] = num + noise < self.p_keep
        df_judgment["score"] = np.where(num + noise < self.p_keep, 1-num-noise, 0)
        return df_judgment

ts1 = DropAndTopSearch("research_1", 0.1, 0.05)
ts2 = DropAndTopSearch("research_2", 0.3, 0.1)
ts3 = DropAndTopSearch("baseline", 0.4, 0.01)
ts4 = DropAndTopSearch("challenger_1", 0.5, 0.02)

search_configs = [ts1, ts2, ts3, ts4]

ts2.run_search(df_judge, str(datetime(2024,9,1))).head(20)

,query,document,judgment,keep,score
0,$30 roblox gift card not digital,B07RX6FBFR,1.00,False,0.00
1,$30 roblox gift card not digital,B09194H44R,0.00,True,0.91
2,$30 roblox gift card not digital,B08R5N6W6B,0.10,False,0.00
3,$30 roblox gift card not digital,B07Y693ND1,0.00,False,0.00
4,$30 roblox gift card not digital,B07RZ75JW3,0.10,True,0.70
5,$30 roblox gift card not digital,B07RZ74VLR,0.10,False,0.00
6,$30 roblox gift card not digital,B07M9XQ9YB,0.00,False,0.00
7,$30 roblox gift card not digital,B078RJ1KZ6,0.00,False,0.00
8,$30 roblox gift card not digital,B01N6RK9UE,0.00,True,0.92
9,$30 roblox gift card not digital,B016Y2BVKA,1.00,False,0.00


## Metrics

### Metrics definitions

Metrics evaluate the performance of a search configuration
To this purpose, they receive a dataframe that contains a search result per row
Metrics functions expect the following columns:
 * keep: whether the result is kept by the search config under evaluation
 * score: the score given to the search result (higher is better)
 * judgment: the judgment value for that row


In [12]:
def dcg(df, sorting_col="score", mask="keep"):
    df = df.sort_values(sorting_col, ascending=False)
    positions = np.arange(df[sorting_col].size) + 1
    dcgs = (2 ** df.judgment - 1) / np.log2(positions+1)
    dcgs = dcgs * df[mask] if mask else dcgs
    return np.sum(dcgs)

def ndcg(df):
    return dcg(df) / dcg(df, "judgment", None)

def precision(df, k):
    df = df.sort_values("score", ascending=False)
    return (df.judgment * df.keep).head(k).sum() / k

def jaccard(df):
    common_count = (df.keep&df.ref_keep).sum()
    union_count = (df.keep|df.ref_keep).sum()
    return common_count / union_count if common_count > 0.0 else common_count

metrics = [
    ("dcg", dcg, None),
    ("ndcg", ndcg, None),
#    ("prec@1", lambda x: precision(x, 1), None),
#    ("prec@5", lambda x: precision(x, 1), None),
    ("prec@10", lambda x: precision(x, 10), None),
#     ("jaccard", jaccard, ts3), # comment out, selecting a reference config is awkward in a pre-calc world
]

### Metrics Calculation

Nested loop over:
 * Search configurations
 * Metrics
 * Judgments dataset (grouped by date and query)

In [13]:
# Create dataset for 7 days
n_days = 7
end_date = datetime(2024,9,1)

df_metrics = []

for search in search_configs:
    for m_name, m_function, ref_search in metrics:
        for x in range(n_days):
            for (query_set_id, query), df_one_query_set in df_query_set.groupby(["query_set_id", "query"]):
                curr_date = end_date - timedelta(days=x)
                queries = df_one_query_set["query"].values
                judgments = df_judge[df_judge["query"]==query]
                ranked_df_sq = search.run_search(judgments, str(curr_date))
                if ref_search:
                    ref_ranked_df_sq = ref_search.run_search(judgments, str(curr_date))
                    ranked_df_sq["ref_keep"] = ref_ranked_df_sq.keep
                    ranked_df_sq["ref_score"] = ref_ranked_df_sq.score
                metric = m_function(ranked_df_sq)
                df_metrics.append(pd.DataFrame({
                    "timestamp": [curr_date],
                    "search_config": [search.name],
                    "query_set_id": [query_set_id],
                    "query": [query],
                    "metric": [m_name],
                    "value": [metric],
                }))
df_metrics = pd.concat(df_metrics)

In [14]:
# Add application and evaluation_id fields
df_metrics["application"] = "sample_data"
df_metrics["evaluation_id"] = df_metrics["search_config"] + "_" + df_metrics["timestamp"].apply(lambda x: str(x))

In [15]:
df_metrics.head()

,datetime,search_config,query_set_id,query,metric,value,application,evaluation_id
0,2024-09-01,research_1,sampled_queries,$30 roblox gift card not digital,dcg,0.00,sample_data,research_1_2024-09-01 00:00:00
0,2024-09-01,research_1,sampled_queries,(fiction without frontiers),dcg,0.00,sample_data,research_1_2024-09-01 00:00:00
0,2024-09-01,research_1,sampled_queries,100,dcg,1.63,sample_data,research_1_2024-09-01 00:00:00
0,2024-09-01,research_1,sampled_queries,15 inch light weight laptop that has lots of m...,dcg,1.05,sample_data,research_1_2024-09-01 00:00:00
0,2024-09-01,research_1,sampled_queries,15 microsoft card,dcg,0.00,sample_data,research_1_2024-09-01 00:00:00


In [16]:
pd.crosstab([df_metrics.query_set_id, df_metrics.search_config], df_metrics.metric, df_metrics.value, aggfunc="mean")

metric                         dcg  ndcg  prec@10
query_set_id    search_config                    
sampled_queries baseline      2.63  0.49     0.49
                challenger_1  3.04  0.56     0.59
                research_1    0.86  0.15     0.12
                research_2    2.35  0.43     0.43
top_queries     baseline      3.04  0.54     0.59
                challenger_1  3.21  0.57     0.63
                research_1    0.93  0.18     0.13
                research_2    2.36  0.42     0.42

In [17]:
# Sometimes baseline has a jaccard of 0 because of the way that fake searches are defined
# (sometimes baseline returns no documents and it messes up the jaccard calculation)

# ix = df_metrics.search_config=="baseline"
# ix = ix & (df_metrics.metric=="jaccard")
# ix = ix & (df_metrics.value < 1)
# df_metrics[ix].head()

### Metrics dataset calculation

The metrics dataset contains the result of evaluating a search config on judgments.
Each row contains the evaluation of a search config on a query on a given metric.

It contains the following fields:
 * datetime: the date and time of the evaluation
 * search_config: search config under evaluation
 * query_set_id: id of the query set used for evaluation
 * query: query from which judgment was derived
 * metric: name of metric used for evaluation
 * value: evaluation result

In [18]:
df_metrics.sample(10)

,datetime,search_config,query_set_id,query,metric,value,application,evaluation_id
0,2024-08-28,baseline,sampled_queries,coil incense,dcg,2.95,sample_data,baseline_2024-08-28 00:00:00
0,2024-09-01,research_2,sampled_queries,mre beef,ndcg,0.71,sample_data,research_2_2024-09-01 00:00:00
0,2024-08-28,research_2,top_queries,ultra fast keto boost 800mg,prec@10,0.17,sample_data,research_2_2024-08-28 00:00:00
0,2024-08-26,challenger_1,sampled_queries,porch shades outdoor roll up,prec@10,0.60,sample_data,challenger_1_2024-08-26 00:00:00
0,2024-08-30,baseline,sampled_queries,columbia ice maiden ii womens insulated snow b...,dcg,0.75,sample_data,baseline_2024-08-30 00:00:00
0,2024-08-26,baseline,sampled_queries,girls bible,ndcg,0.54,sample_data,baseline_2024-08-26 00:00:00
0,2024-08-31,challenger_1,sampled_queries,the gallows,prec@10,0.63,sample_data,challenger_1_2024-08-31 00:00:00
0,2024-08-30,research_1,sampled_queries,collagen supplements,prec@10,0.10,sample_data,research_1_2024-08-30 00:00:00
0,2024-08-28,research_2,sampled_queries,black and white checkered curtains 2 panels,prec@10,0.31,sample_data,research_2_2024-08-28 00:00:00
0,2024-09-01,research_2,sampled_queries,rudin by ivan turgenev,prec@10,0.52,sample_data,research_2_2024-09-01 00:00:00


In [19]:
_16.iloc[1:5]

metric                         dcg  ndcg  prec@10
query_set_id    search_config                    
sampled_queries challenger_1  3.04  0.56     0.59
                research_1    0.86  0.15     0.12
                research_2    2.35  0.43     0.43
top_queries     baseline      3.04  0.54     0.59

In [20]:
df_metrics.to_csv("metrics.csv", index=False)

# Save to json format for bulk ingestion

We save only the top queries to avoid generating too large a bulk request.

In [21]:
with open("sample_data.ndjson", "w") as f:
    for _, metric in df_metrics[df_metrics.query_set_id=="top_queries"].iterrows():
        metric_dict = {
            "timestamp": metric["timestamp"].isoformat(),
            "search_config": metric["search_config"],
            "query_set_id": metric["query_set_id"],
            "query": metric["query"],
            "metric": metric["metric"],
            "value": metric["value"],
            "application": metric["application"],
            "evaluation_id": metric["evaluation_id"],
        }
        event_id = str(uuid.uuid4())
        f.write(json.dumps(
            {"index": {"_index": "sqe_metrics_sample_data", "_id": event_id}}
        ) + "\n")
        f.write(json.dumps(metric_dict) + "\n")
